In [1]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import exists

def infer_column_types(df):
    for col in df.columns:
        try:
            unique_values = df[col].dropna().unique()
        except:
            df[col] = df[col].apply(lambda x: json.dumps(x))
            unique_values = df[col].dropna().unique()
        
        if set(unique_values).issubset({"True", "False", "0", "1"}):
            df[col] = df[col].map(lambda x: True if x in ["True", "1"] else False).astype("bool")
        elif np.all(~pd.isna(pd.to_numeric(unique_values, errors="coerce"))): 
            df[col] = pd.to_numeric(df[col], errors="coerce", downcast="integer")
        elif df[col].nunique() / len(df) < 0.1:
            df[col] = df[col].astype("string").astype("category")



In [2]:
os.makedirs(f"helm_tables", exist_ok=True)
BENCHMARKS = ["air-bench", "classic", "thaiexam", "mmlu", "lite"]
lo = lambda x: json.load(open(x, "r"))

task2metric = lo("task2metric.json")
task2metric = pd.json_normalize(task2metric)

all_paths = []
for benchmark in BENCHMARKS:
    dir_path = f"helm_jsons/{benchmark}/releases"
    assert exists(dir_path)
    latest_release = sorted(os.listdir(dir_path))[-1]
    folder_dict = lo(f"{dir_path}/{latest_release}/runs_to_run_suites.json")
    all_paths += [f"helm_jsons/{benchmark}/runs/{s}/{r}" for r, s in folder_dict.items()]

files = ["display_requests.json", "display_predictions.json", "run_spec.json"]
all_paths = [p for p in tqdm(all_paths) if all([exists(f"{p}/{f}") for f in files])]
all_lists = [[lo(f"{p}/{f}") for p in tqdm(all_paths)] for f in files]


100%|██████████| 15815/15815 [00:00<00:00, 29187.57it/s]


In [ ]:
results = []
for d_requests, d_predictions, run_specs, paths in tqdm(zip(*all_lists, all_paths), total=len(all_lists[0])):
    d_requests = pd.json_normalize(d_requests)
    d_predictions = pd.json_normalize(d_predictions)
    run_specs = pd.json_normalize(run_specs)
    benchmark = paths.split("/")[1]
    run_specs["benchmark"] = benchmark
    run_specs = run_specs.loc[run_specs.index.repeat(d_predictions.shape[0])].reset_index(drop=True)
    overlap_column = d_predictions.columns.intersection(d_requests.columns)
    d_requests = d_requests.drop(columns=overlap_column)
    result = pd.concat([d_requests, d_predictions, run_specs], axis=1)
    result["scenario"] = result['name'].str.split(r'[:,]', n=1, expand=True)[0]
    result["scenario"] = result["scenario"].astype("category")
    assert result["scenario"].nunique() == 1
    metric_name = task2metric[f"{benchmark}.{result['scenario'].iloc[0]}"].iloc[0]
    if isinstance(metric_name, list):
        for metric_name_ in metric_name:
            dicho_score = result.get(f"stats.{metric_name_}", pd.NA)
            if dicho_score is not pd.NA:
                if not dicho_score.isna().all():
                    result["dicho_score"] = dicho_score
                    break
    else:
        result["dicho_score"] = result.get(f"stats.{metric_name}", pd.NA)
    results.append(result)

results = pd.concat(results, axis=0, join='outer')
infer_column_types(results)
results.reset_index(drop=True, inplace=True)
for col in results.columns:
    if results[col].dtype != "category" and np.isnan(results[col]).all():
        results = results.drop(columns=col)
    else:
        if results[col].dtype == "float64" and np.nanmax(results[col]) < 65500 and np.nanmin(results[col]) > -65500:
            results[col] = results[col].astype("float16")
results.to_pickle("helm_tables/responses.pkl")

100%|██████████| 15815/15815 [11:37<00:00, 22.69it/s]  
/tmp/user/21130/ipykernel_2666996/2641024592.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat(results, axis=0, join='outer')
